In [7]:
import cv2
import numpy as np
from keras.models import model_from_json
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

class EmotionDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Emotion Detection App")

        
        self.cap = cv2.VideoCapture(0)
        
        self.face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


        json_file = open("D:/Emotion_detection_with_CNN-main/model/emotion_model.json", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.emotion_model = model_from_json(loaded_model_json)
        self.emotion_model.load_weights("D:/Emotion_detection_with_CNN-main/model/emotion_model.h5")

    
        self.label = tk.Label(root, text="Emotion Detection", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.canvas = tk.Canvas(root, width=self.cap.get(cv2.CAP_PROP_FRAME_WIDTH), height=self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.canvas.pack()

        self.quit_button = ttk.Button(root, text="Quit", command=self.quit_app)
        self.quit_button.pack(pady=10)

        
        self.emotion_detection_loop()

    def quit_app(self):
        self.cap.release()
        self.root.destroy()

    def emotion_detection_loop(self):
        ret, frame = self.cap.read()
        if ret:
            #frame=cv2.resize(frame,(1280,720))
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            col = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            num_faces = self.face_detector.detectMultiScale(
                col,
                scaleFactor=1.5,
                minNeighbors=5,
                minSize=(50, 60),
            )

            for (x, y, w, h) in num_faces:
                cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (0, 255, 0), 4)
                roi_gray_frame = gray_frame[y:y + h, x:x + w]
                cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

                emotion_prediction = self.emotion_model.predict(cropped_img)
                maxindex = int(np.argmax(emotion_prediction))
                cv2.putText(frame, emotion_dict[maxindex], (x + 5, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2,
                            cv2.LINE_AA)

            self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

        self.root.after(10, self.emotion_detection_loop)  


if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionDetectionApp(root)
    root.mainloop()


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 28ms/step
